# Задание

В базе данных есть следующие таблицы:
* ***city***: city_id; client_city_id; city; client_city
* ***clients***: client_id; client_city_id; birth_date; registration
* ***promotion***: promotion_id; category_id; promotion_name; category_name; partner_id; partner_name
* ***purchase***: purchase_id; partner_id; client_id; city_id; promotion_id; category_id; purchase_date; price; quantity; status

Необходимо написать запрос, чтобы получить таблицу со следующими столбцами:
* ***purchase_date*** – дата покупки;
* ***purchase_id*** – id покупки;
* ***client_id*** – id покупателя;
* ***client_age*** – возраст покупателя;
* ***client_registration_age*** – как долго человек пользуется вашими магазинами;
* ***client_category*** – содержит new или old, если это первая или последующая покупка соответственно;
* ***promotion_name*** – акция;
* ***category_name*** – категория;
* ***partner_name*** – партнёр;
* ***client_city*** – город, где находится покупатель;
* ***city*** – город;
* ***revenue*** – сумма выручки;
* ***quantity*** – число проданных единиц

Дополнительные условия:
* В таблице должны присутствовать только значения, где в поле status стоит значение 1.
* И те значения, где purchase_date находится в диапазоне от 01.05.2020 до 01.08.2020.

In [19]:
import pandahouse as ph

In [20]:
connection_test = dict(database='test',
                  host='http://clickhouse.beslan.pro:8080',
                  user='student-rw',
                  password='656e2b0c9c')

In [21]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'test',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

### Структура исходных таблиц

In [22]:
city_q = """
        SELECT *
        FROM test.t_city
        LIMIT 5
        """

In [23]:
clients_q = """
        SELECT *
        FROM test.t_clients
        LIMIT 5
        """

In [24]:
promotion_q = """
        SELECT *
        FROM test.t_promotion
        LIMIT 5
        """

In [25]:
purchase_q = """
        SELECT *
        FROM test.t_purchase
        LIMIT 5
        """

In [26]:
city = ph.read_clickhouse(query=city_q, connection=connection_default)
clients = ph.read_clickhouse(query=clients_q, connection=connection_default)
promotion = ph.read_clickhouse(query=promotion_q, connection=connection_default)
purchase = ph.read_clickhouse(query=purchase_q, connection=connection_default)

In [27]:
city

,city_id,client_city_id,city,client_city
0,1,4,Fort Saskatchewan,Bonlez
1,2,5,Lakeland County,Cassaro
2,3,6,West Valley City,Ovalle
3,4,7,Argyle,Maidenhead
4,5,8,Calarcá,Aisén


In [28]:
clients

,client_id,client_city_id,birth_date,registration
0,1,57,1978-02-23 18:29:41,2019-12-09 21:56:30
1,2,25,1978-10-12 06:52:13,2020-11-03 00:12:19
2,3,37,1988-03-20 16:27:32,2021-09-09 18:44:12
3,4,41,1979-09-15 02:02:06,2019-10-29 10:42:54
4,5,76,2005-09-02 23:36:25,2020-08-29 15:38:45


In [29]:
promotion

,promotion_id,category_id,partner_id,promotion_name,category_name,partner_name
0,1,1,1,south,food,new1
1,1,1,2,south,food,new2
2,1,1,3,south,food,new3
3,1,1,4,south,food,new4
4,2,1,5,north,food,new5


In [30]:
purchase

,purchase_id,partner_id,client_id,city_id,promotion_id,category_id,price,quantity,purchase_date,status
0,1,5,14,35,1,1,840,7,04.12.2019 21:47,Yes
1,2,3,44,30,1,1,764,27,09.10.2019 16:04,Yes
2,3,14,35,25,1,1,573,3,29.05.2020 22:00,Yes
3,4,17,49,31,1,1,554,30,22.06.2020 22:27,No
4,5,8,17,59,1,1,582,28,24.03.2020 13:47,No


### SQL-запрос для получения финальной таблицы

In [34]:
q = """    
    SELECT 
        purchase_date,
        purchase_id,
        client_id,
        client_age,
        client_registration_age,
        client_category,
        promotion_name,
        category_name,
        partner_name,
        client_city,
        city,
        revenue,
        quantity
    FROM
        (
        SELECT
            purchase_date,
            purchase_id,
            client_id,
            promotion_name,
            client_category,
            category_name,
            partner_name,
            (price * quantity) AS revenue,
            quantity,
            partner_id
        FROM
            (
            SELECT
                purchase_id,
                partner_id,
                client_id,
                price,
                quantity,
                purchase_date,
                status,
                CASE
                    WHEN client_category='new' THEN 'new'
                    ELSE 'old'
                END AS client_category
            FROM 
                (
                SELECT 
                    purchase_id,
                    partner_id,
                    client_id,
                    price,
                    quantity,
                    parseDateTimeBestEffort(purchase_date) AS purchase_date,
                    status
                FROM test.t_purchase
                ) AS l
            FULL JOIN
                (
                WITH (SELECT MIN(parseDateTimeBestEffort(purchase_date))
                FROM test.t_purchase) AS min_date

                SELECT 
                    client_id,
                    first_purchase,
                    CASE
                        WHEN first_purchase>=min_date THEN 'new'
                        ELSE 'old'
                    END AS client_category
                FROM
                    (
                    SELECT 
                        client_id,
                        MIN(parseDateTimeBestEffort(purchase_date)) AS first_purchase
                    FROM
                        test.t_purchase
                    GROUP BY client_id
                    )
                ) 
            AS r
                ON l.client_id=r.client_id AND l.purchase_date=r.first_purchase
                HAVING 
                    status = 'Yes' 
                    AND toDate(purchase_date) >= '2020-05-01' 
                    AND toDate(purchase_date) <= '2020-08-01'
            ) AS l
        JOIN 
            (
            SELECT
                partner_id,
                promotion_name,
                category_name,
                partner_name
            FROM test.t_promotion
            )
        AS r
            ON l.partner_id=r.partner_id
        ) AS l
    JOIN 
        (
        SELECT
            client_id,
            dateDiff('year', toDateTime(birth_date), today()) AS client_age,
            dateDiff('day', toDateTime(registration), today()) AS client_registration_age,
            client_city,
            client_city_id,
            city
        FROM test.t_clients AS l
        JOIN
            (
            SELECT
                client_city,
                client_city_id,
                city
            FROM test.t_city
            )
        AS r
            ON l.client_city_id=r.client_city_id
        )
    AS r
        ON l.client_id=r.client_id
    HAVING client_registration_age >= 0
"""

In [35]:
df = ph.read_clickhouse(query=q, connection=connection_default)

In [36]:
df.head()

,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-05-13 07:15:00,12,10,34,284,old,north,food,new5,Sevsk,Krishnanagar,23409,27
1,2020-05-13 18:30:00,20,45,30,243,old,west,clothes,new11,Sant'Eufemia a Maiella,Aserrí,6591,13
2,2020-07-20 03:36:00,32,40,15,201,old,south,food,new1,Hofstade,Göttingen,6195,7
3,2020-06-06 13:51:00,33,30,24,381,old,east,clothes,new17,Baltimore,Zapopan,8862,14
4,2020-07-08 00:26:00,47,22,39,198,new,east,clothes,new17,Melton Mowbray,Delianuova,17982,18


### Для получения необходимой таблицы были сделаны следующие шаги:
* на основе таблицы purchase была создана колонка client_category: сначала путем агрегации я нашла первую дату покупки каждого клиента и через CASE добавила туда колонку client_category с записью 'new', затем объединила через FULL JOIN эту таблицу с изначальной purchase и заполнила пропуски в client_category как 'old';
* на основе таблицы client созданы столбцы client_age (разница между текущей датой и днем рождения клиента) client_registration_age (разница между текущей датой и датой регистрации клиента);
* на основе таблицы purchase создан столбец revenue как произведение стоимости товара на количество;
* объединены таблицы purchase и promotion по переменной partner_id; объединены таблицы client и city по переменной client_city_id; получившиеся две таблицы объединены между собой по переменной client_id